In [ ]:
import glob
import os

import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import pandas as pd

import torch.nn as nn
from torchvision import models

from albumentations.pytorch import ToTensorV2, ToTensor
import albumentations as alb

import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [ ]:
from torch.cuda.amp import autocast


class EfficientNet(nn.Module):
    def __init__(self, n_classes, pretrained_backbone, mixed_precision, model_name='tf_efficientnet_b7_ns'):
        super().__init__()
        self.amp = mixed_precision
        self.model = timm.create_model(model_name, pretrained=pretrained_backbone)
        n_features = self.model.classifier.in_features
        self.model.global_pool = nn.Identity()
        self.model.classifier = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(n_features, 11)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        if self.amp:
            with autocast():
                bs = x.size(0)
                features = self.model(x)
                pooled_features = self.pooling(features).view(bs, -1)
                x = self.classifier(pooled_features)
        else:
            bs = x.size(0)
            features = self.model(x)
            pooled_features = self.pooling(features).view(bs, -1)
            x = self.classifier(pooled_features)
        return self.sigmoid(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EfficientNet(11, pretrained_backbone=False, mixed_precision=True, model_name='tf_efficientnet_b7_ns')

state_dict = torch.load('../input/effnet7-epoch14-val-auc-0947-loss-0455/model_epoch_14_val_auc_0.947_loss_0.455_train_auc_0.951_loss_0.34.pth')
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)
model.eval()
model = model.to(device)
model = model.float()

In [ ]:
    def __init__(self, transform, dataset_filepath, image_h_w_ratio=0.8192, width_size=128):
        self.files = [filepath for filepath in glob.iglob(os.path.join(dataset_filepath, '*'))]
        self.transform = transform
        self.dataset_filepath = dataset_filepath
        self.image_h_w_ratio = image_h_w_ratio
        self.width_size = width_size
        self.height_size = int(self.image_h_w_ratio * self.width_size)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image_filepath = self.files[idx]
        image = cv2.imread(image_filepath)

        image_h, image_w = image.shape[0], image.shape[1]
        if image_h / image_w > self.image_h_w_ratio:
            ratio_coeff = self.height_size / image_h
        else:
            ratio_coeff = self.width_size / image_w
        new_h = int(image_h * ratio_coeff)
        new_w = int(image_w * ratio_coeff)
        image = cv2.resize(image, (new_w, new_h))

        w_padding = (self.width_size - new_w) / 2
        h_padding = (self.height_size - new_h) / 2
        l_padding = int(w_padding)
        t_padding = int(h_padding)
        r_padding = int(new_w + (w_padding if w_padding % 1 == 0 else w_padding - 0.5))
        b_padding = int(new_h + (h_padding if h_padding % 1 == 0 else h_padding - 0.5))

        result_image = np.full((self.height_size, self.width_size, 3), 0, dtype=np.uint8)
        result_image[t_padding:b_padding, l_padding:r_padding, :] = image
        result_image = np.reshape(result_image, (result_image.shape[0], result_image.shape[1], 3))

        if self.transform:
            result_image = self.transform(image=result_image)

        return image_filepath.rsplit('/', 1)[1].rsplit('.', 1)[0], result_image['image']

In [ ]:
image_transforms = alb.Compose([
    alb.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])
test_set = ImageDataset(image_transforms, '../input/ranzcr-clip-catheter-line-classification/test', width_size=640)
test_loader = DataLoader(test_set, batch_size=64, num_workers=0, pin_memory=True)

In [ ]:
predictions = []
filepaths = []

with torch.no_grad():
    for batch in test_loader:
        filepaths_batch, image_batch = batch
        prediction_batch = model(image_batch.to(device))

        predictions.extend(prediction_batch.cpu().numpy())
        filepaths.extend(filepaths_batch)

In [ ]:
predictions = np.array(predictions)
df = pd.DataFrame({
    'StudyInstanceUID': filepaths,
    'ETT - Abnormal': predictions[:, 0],
    'ETT - Borderline': predictions[:, 1],
    'ETT - Normal': predictions[:, 2],
    'NGT - Abnormal': predictions[:, 3],
    'NGT - Borderline': predictions[:, 4],
    'NGT - Incompletely Imaged': predictions[:, 5],
    'NGT - Normal': predictions[:, 6],
    'CVC - Abnormal': predictions[:, 7],
    'CVC - Borderline': predictions[:, 8],
    'CVC - Normal': predictions[:, 9],
    'Swan Ganz Catheter Present': predictions[:, 10]})
df.to_csv('/kaggle/working/submission.csv', index=False)